# Woolsey Fire Data Exploration

This notebook reviews satellite and model data related to the 2018 Woolsey Fire, including vegetation, climate, and topographic data.

In [ ]:
import os
from pathlib import Path
import warnings

import earthpy as et
import earthpy.plot as ep
import geopandas as gpd
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rasterio.enums import Resampling
import rioxarray as rxr
import xarray as xr

from ea_drought_burn.config import CRS
from ea_drought_burn.utils import (
    copy_xr_metadata,
    load_nifc_fires,
    open_raster,
    plot_bands,
    plot_rgb,
    reproject_match
)
from ea_drought_burn.notebooks import run_notebook




# Set default plotting parameters
plt.rc("font", size=18)
plt.rc("axes", labelsize=18)
plt.rc("xtick", labelsize=18)
plt.rc("ytick", labelsize=18)
plt.rc("legend", fontsize=18)
plt.rc("figure", titlesize=24)
plt.rc("figure.constrained_layout", use=True, h_pad=12/72, w_pad=12/72)

# Set working directory to the earthpy data directory
os.chdir(os.path.join(et.io.HOME, "earth-analytics", "data", "woolsey-fire"))

In [ ]:
def plot_four(
    bands,
    boundary,
    mask="{}",
    width=16,
    suptitle=None,
    years=(2013, 2014, 2015, 2016),
    **kwargs
):
    """Plots bands in two columns"""
    
    # Scale fonts
    font_size = plt.rcParamsDefault["font.size"]
    plt.rc("font", size=font_size * width // 16)
    
    # Scale outline
    bound_kwargs = {"edgecolor": "black", "facecolor": "none", "linewidth": 2}
    bound_kwargs["linewidth"] *= width // 16
    
    titles = [mask.format(year) for year in years]
    
    nrows = int(np.ceil(len(bands) / 2))
    height = (width * 0.7) / 2 * nrows

    fig, axes = plt.subplots(nrows, 2, figsize=(width, height))
    
    if suptitle:
        fig.suptitle(suptitle)
    
    axes_flat = []
    for row in axes:
        try:
            axes_flat.extend(row)
        except TypeError:
            axes_flat.append(row)
          
    # Delete last axis if not an even number of bands
    if len(bands) % 2:
        axes_flat[-1].remove()

    for ax, title, band in zip(axes_flat, titles, bands):
        plot_bands(band, ax=ax, title=title, **kwargs)
        boundary.plot(ax=ax, **bound_kwargs)
    
    # Reset font size to default
    plt.rcParams.update(plt.rcParamsDefault)

## Restore and prepare data

To ensure consitency across notebooks and avoid repeating code, all data is loaded using **load_data.ipynb** and restored as needed in individual notebooks. Please see that notebook for details about the sources of data used here.

In [ ]:
# Check for stored ready variable and run load-data.ipynb if not found
%store -r woolsey_data_ready
try:
    woolsey_data_ready
except NameError:
    print("Running load-data.ipynb...")
    run_notebook("load-data.ipynb")

# Retore variables using storemagic. Each variable is restored explicitly to
# avoid confusion about where variable names are coming from.
%store -r all_data
%store -r cmap_dnbr
%store -r labels_vegetation
%store -r labels_dnbr
%store -r cmap_vegetation
%store -r prism_grid
%store -r reproj_to
%store -r woolsey_fire

# Set crop boundary
crop_bound = woolsey_fire.envelope

## USGS topographic data

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 16))
fig.suptitle("Woolsey Fire - Topography - USGS - 10 m")
plot_bands(all_data["Elevation"], ax=ax1, title="Elevation")
plot_bands(all_data["Slope"], ax=ax2, title="Slope")
plot_bands(all_data["Aspect"], ax=ax3, title="Aspect")
plot_bands(all_data["Folded Aspect"], ax=ax4, title="Folded aspect")

pass

**Description:** Topographic data for the Woolsey Fire scar, including elevation, slope, aspect, and folded aspect. Folded aspect. calculated as *|180 - |aspect - 225||*, converts aspect into a linear parameter where hotness/dryness increases moving from 0Â° (for a northeast-facing slope) to 180Â° (for a southwest-facing slope) (McCune and Keon, 2002).

## Sentinel-2 pre- and post-fire images

Pre- and post-fire Sentinel-2 imagery is much better than the available Landsat images, which are obscured by smoke or clouds through early 2019.

In [ ]:
# Plot the pre- and post-fire images
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
fig.suptitle("Woolsey Fire - Sentinel-2 imagery - 30 m")

# Plot pre-fire image
plot_rgb(all_data["Sentinel-2 Prefire"], 
         ax=ax1,
         rgb=[3, 2, 1],
         stretch=True, title="2018-10-31")

# Plot post-fire image
plot_rgb(all_data["Sentinel-2 Postfire"],
         ax=ax2,
         rgb=[3, 2, 1],
         stretch=True,
         title="2018-12-15")

# Plot Woolsey Fire boundary
for ax in (ax1, ax2, ax3, ax4):
    woolsey_fire.plot(ax=ax, edgecolor="white", facecolor="none", linewidth=2)

In [ ]:
# Plot pre-fire Sentinel-2 spectral indides
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 16))
fig.suptitle("Woolsey Fire - Sentinel-2 spectral indices - 30 m\n"
             "31 Oct 2018 (pre-fire)")

plot_bands(all_data["NDVI"],
           ax=ax1,
           cmap="RdYlGn",
           vmin=-1,
           vmax=1,
           title="NDVI")

plot_bands(all_data["NDWI"],
           ax=ax2,
           cmap="RdYlGn",
           vmin=-1,
           vmax=1,
           title="NDWI")

plot_bands(all_data["NDMI"],
           ax=ax3,
           cmap="RdYlGn",
           vmin=-1,
           vmax=1,
           title="NDMI")

plot_bands(all_data["SAVI"],
           ax=ax4,
           cmap="RdYlGn",
           vmin=-1,
           vmax=1,
           title="SAVI")

# Plot Woolsey Fire boundary
for ax in (ax1, ax2, ax3, ax4):
    woolsey_fire.plot(ax=ax, edgecolor="white", facecolor="none", linewidth=2)

**Description:** NDMI and NDWI show a pattern broadly similar to the one seen in the FAL data.

In [ ]:
# Calculate dNBR from Sentinel-2 data
nir = all_data["Sentinel-2 Prefire"][8]
swir2 = all_data["Sentinel-2 Prefire"][12]
pre_nbr = (nir - swir2) / (nir + swir2)

nir = all_data["Sentinel-2 Postfire"][7]
swir2 = all_data["Sentinel-2 Postfire"][11]
post_nbr = (nir - swir2) / (nir + swir2)

s2_dnbr = pre_nbr - post_nbr

# Calculate classified dNBR from Sentinel-2 data
bins = [-np.inf, -.1, .1, .27, .66, np.inf]
s2_cl_dnbr = xr.apply_ufunc(np.digitize, s2_dnbr, bins)

s2_cl_dnbr = s2_cl_dnbr.astype(np.float64)
s2_cl_dnbr = s2_cl_dnbr.rio.write_nodata(np.nan)
s2_cl_dnbr = s2_cl_dnbr.rio.clip(woolsey_fire.geometry)

# Reorder categories to match MTBS data
s2_cl_dnbr -= 1
s2_cl_dnbr = s2_cl_dnbr.where((s2_cl_dnbr >= 0) | ~np.isfinite(s2_cl_dnbr), 4)

In [ ]:
# Plot Sentinel-2 dNBR and classified dNBR
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

plot_bands(s2_dnbr,
           ax=ax1,
           cmap="RdYlGn_r",
           vmin=all_data["dNBR"].min(),  # use MTBS min so plots are comparable
           vmax=all_data["dNBR"].max(),  # use MTBS max so plots are comparable
           title=("Woolsey Fire - Sentinel dNBR - 30 m\n"
                  "31 Oct 2018 - 15 Dec 2018"))

plot_bands(s2_cl_dnbr,
           ax=ax2,
           cmap=cmap_dnbr,
           vmin=1,
           vmax=5,
           title=("Woolsey Fire - Sentinel-2 classified dNBR - 30 m\n"
                  "31 Oct 2018 - 15 Dec 2018"),
           cbar=False,
           scale=False)

# Draw legend
ep.draw_legend(im_ax=ax2.get_images()[0],
               classes=range(5),
               titles=labels_dnbr,
               bbox=(0.01, 0.99))

# Plot Woolsey Fire boundary
for ax in (ax1, ax2):
    woolsey_fire.plot(ax=ax, edgecolor="black", facecolor="none", linewidth=2)

**Description:** The Sentinel-2 burn-severity plots show a similar pattern to the MTBS plots below but have generally higher burn severities, including a larger number of high-severity pixels and a smaller number of unburned pixels.

## MTBS burn severity plots

MTBS provides normalized, field-validated dNBR plots that are more reliable than those that, like the one I calculated above, are made solely from satellite images.

In [ ]:
# Plot MTBS dNBR and classified dNBR
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

plot_bands(all_data["dNBR"],
           ax=ax1,
           cmap="RdYlGn_r",
           title=("Woolsey Fire - MTBS dNBR - 30 m\n"
                  "Dec 15, 2017 - Dec 15, 2018"))

plot_bands(all_data["Classified dNBR"],
           ax=ax2,
           cmap=cmap_dnbr,
           vmin=1,
           vmax=5,
           title=("Woolsey Fire - MTBS classified dNBR - 30 m\n"
                  "Dec 15, 2017 - Dec 15, 2018"),
           cbar=False,
           scale=False)

# Draw legend
ep.draw_legend(im_ax=ax2.get_images()[0],
               classes=range(5),
               titles=labels_dnbr,
               bbox=(0.01, 0.99))

# Plot Woolsey Fire boundary
for ax in (ax1, ax2):
    woolsey_fire.plot(ax=ax, edgecolor="black", facecolor="none", linewidth=2)

## Woolsey Fire Stack

The original data included a second stack, WoolseyFireDataStack2, that included some Landsat data and BARC burn severity data, as well as the FAL data included in the SMM stack. All of that content is replicated elsewhere, and this file is not included in the final version of the data for this project.

+ **Landsat:** The WoolseyFireDataStack2 file contains two sets of Landsat files,
  one from 2017-10-24 and one from 2018-10-18. Both pre-date the Woolsey fire and
  include only bands 4 (red) and 7 (SWIR2). None of the spectral indices I'm aware
  of use those two bands.
+ **BARC:** BARC stands for Burned Area Reflectance Classification and is a 
  burn-severity index similar to dNBR. We decided to use the MTBS dNBR data
  instead.

## Santa Monica Mountain Stack

The Santa Monica Mountain (SMM) stack includes data about the vegetation and climate of the Santa Monica Mountain region in 2013-2016.

In [ ]:
# Plot vegetation community

# Figure is large, so increase font size
plt.rc("font", size=24)

# Make plot
fig, ax = plt.subplots(figsize=(20, 20))
plot_bands(all_data["Community"],
           ax=ax,
           title="SMM vegetation community map",
           cmap=cmap_vegetation,
           cbar=False)
woolsey_fire.plot(ax=ax, edgecolor="black", facecolor="none", linewidth=4)

# Draw legend. This function forces the font size to 13, so the plot canvas
# can only be so big before the legend becomes illegible.
legend = ep.draw_legend(ax.get_images()[0], 
               titles=labels_vegetation,
               classes=range(len(labels_vegetation)))

# Restore original font size
plt.rc("font", size=18)

**Description:** Many of the salients into and low-severity burn areas within the fire scar are classified as substrate. Moderate- and high-severity burns appear mostly in the chapparal-dominated southern half of the scar.

### Fraction alive

The fraction-alive metric (FAL or RFAL) measures the ratio of healthy vegetation (green) to total vegetation (green plus brown). Living vegetation on the 2016 FAL plot has a similar distribution to the areas of highest burn severity during the Woolsey Fire, which was one impetus for this project. FAL data is only available through 2016, which leaves a two-year gap between the last snapshot of vegetation health and the Woolsey Fire, but pre-fire spectral indices like NDWI show a similar pattern.

In [ ]:
# Plot fraction alive. The FAL plots are all huge because smaller plots have
# large white patches that look to me like a problem with interpolating NaNs.
plot_four(all_data["FAL"],
          woolsey_fire,
          width=64,
          suptitle="Fraction Alive (FAL)",
          cmap="PiYG",
          vmin=0,
          vmax=1)

#### dFAL

The year-on-year difference in fraction alive was calculated as dFAL. I calculated this index by subtracting the FAL of the previous year from the current year, so a positive dFAL means that the fraction alive has increased and a negative dFAL means that fraction alive has decreased.

In [ ]:
# Plot year-on-year difference in fraction alive
plot_four(all_data["dFAL"],
          woolsey_fire,
          width=64,
          suptitle="Change in Fraction Alive (dFAL)",
          years=("2013-2014", "2014-2015", "2015-2016"),
          cmap="PiYG",
          vmin=-1,
          vmax=1)

#### Dead pixels and the dieback threshold

Foster et al. (2017) use a threshold of 0.5431 to distinguish dead pixels from
live pixels based on a comparison of FAL and a field survey of oaks. The
correlation between the aerial and field surveys is poor and only looked at one
of the six vegetation communities in the study area, so that threshold is
poorly constrained. I've used a threshold of 0.5 below to identify dead pixels.

In [ ]:
# Plot dead pixels
plot_four(all_data["Dead"],
          woolsey_fire,
          width=64,
          suptitle="Dead Pixels (FAL < 0.5)",
          cmap="Greys",
          cbar=False)

#### Years dead

Using the same threshold as above, the years-dead plot tries to quantify how long pixels have been dead for (that is, how long FAL has been less than 0.5). Years are given relative to 2018, although FAL data for 2017 and 2018 is not available. This calculation has some semantic problems at the other edge, too: Anything classified as dead in 2013 was assigned a value of 4, but those pixels could have been that way for more than four years.

In [ ]:
# Figure is large, so increase font size
plt.rc("font", size=24)

cmap = ListedColormap(["green", "lightgreen", "yellow", "tan", "brown", "red"])

fig, ax = plt.subplots(figsize=(20, 20))
plot_bands(all_data["Years Dead"],
           ax=ax,
           title="Years Dead (FAL < 0.5)",
           cmap=cmap,
           cbar=None)

woolsey_fire.plot(ax=ax, edgecolor="black", facecolor="none", linewidth=4)

# Draw legend. This function forces the font size to 13, so the plot canvas
# can only be so big before the legend becomes illegible.
ep.draw_legend(ax.get_images()[0], 
               titles=[str(i) for i in range(6)],
               classes=range(6))

# Restore original font size
plt.rc("font", size=18)

#### Area lost

In [ ]:
# Calculate area lost per vegetation type
# FIXME: Check this, numbers differ from previous work

print(f"Total area:   {2089 * 1681 * 15.6 **2 / 1e6:.0f} km2")
print("--")

comm = all_data["Community"]
dfal_13_16 = all_data["FAL"][3] - all_data["FAL"][0]
for i in range(0, 7):
    subset = xr.where(comm == i, True, False).values
    
    # Limit to vegetation (i.e., pixels with FAL calculated)
    comm_13_16 = dfal_13_16.where(subset)
    
    dfal = comm_13_16.mean().item()
    pixels = xr.where(comm_13_16, 1, 0).sum().item()
    pixel_km2 = comm_13_16.rio.resolution()[0] ** 2 * 1e-6
    
    area_km2 = dfal * pixels * pixel_km2
    
    print(labels_vegetation[i])
    print(f"Pixels:          {pixels}")
    print(f"Area (km2):      {pixels * pixel_km2:.1f} km2")
    print(f"Area (acres):    {pixels * pixel_km2 / 0.004:.1f} acres")
    print(f"Dieback %:       {dfal:.3f}")
    print(f"Dieback (km2):   {area_km2:.1f} km2")
    print(f"Dieback (acres): {area_km2 / 0.004:.1f} acres")
    print("--")

### Climate data

The SMM stack includes climate data from the PRISM project. In the plots below, green can generally be understood as being wetter/cooler and yellow as hotter/drier. Colors in each set of plots are scaled to the same range.

The highest, driest conditions in the area burned during the Woolsey Fire generally occurred in the northeastern part of the scar.

#### Days with precipitation over 0.1 inchdes

In [ ]:
# Plot number of days with precipiation over 0.1 inches
key = "Days Precipitation"
xda = all_data[key]

plot_four(xda,
          woolsey_fire,
          suptitle=key,
          cmap="summer_r",
          vmin=xda.min(),
          vmax=xda.max())

#### Maximum vapor pressure deficit

The vapor pressure deficit (VPD) measures how much water would have to added to the air before it would be saturated. High values indicate drier conditions.

In [ ]:
# Plot maximum VPD
key = "Max VPD"
xda = all_data[key]

plot_four(xda,
          woolsey_fire,
          suptitle=key,
          cmap="summer",
          vmin=xda.min(),
          vmax=xda.max())

#### Minimum temperature

In [ ]:
# Plot minimum temperatue
key = "Minimum Temperature"
xda = all_data[key]

plot_four(xda,
          woolsey_fire,
          suptitle=key,
          cmap="summer_r",
          vmin=xda.min(),
          vmax=xda.max())

#### Heat days over 95 Â°F

In [ ]:
# Plot number of days over 95 F
key = "Heat Days Over 95"
xda = all_data[key]

plot_four(xda,
          woolsey_fire,
          suptitle=key,
          cmap="summer",
          vmin=xda.min(),
          vmax=xda.max())

#### Cumulative precipitation

In [ ]:
# Plot cumulative precipitation
key = "Cumulative Precipitation"
xda = all_data[key]

plot_four(xda,
          woolsey_fire,
          suptitle=key,
          cmap="summer_r",
          vmin=xda.min(),
          vmax=xda.max())

## Previous fires

The area around the Woolsey Fire suffered ~24 wildfires between 2000 and 2018. Two of those fires overlap with unburned patches with the fire scar. This section summarizes the recent fire history and shows where that history may have impacted the severity of the Woolsey Fire.

In [ ]:
# Create plot comparing recently burned areas to dNBR
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

plot_bands(all_data["Classified dNBR"],
           ax=ax1,
           cmap=cmap_dnbr,
           vmin=1,
           vmax=5,
           title=("Woolsey Fire - MTBS classified dNBR - 30 m\n"
                  "Dec 15, 2017 - Dec 15, 2018"),
           cbar=False,
           scale=False)

plot_bands(all_data["Burned (2000-2018)"],
           ax=ax2,
           title=("Recent fires in the vicinity of the Woolsey Fire - NIFC\n"
                  "2000-2018"))

woolsey_fire.plot(ax=ax1, edgecolor="black", facecolor="none", linewidth=2)
woolsey_fire.plot(ax=ax2, edgecolor="white", facecolor="none", linewidth=2)

pass

**Description:** MTBS dNBR plot (left) and areas of recent wildfires (right). Two areas dark green areas on the dNBR plot--one in the southeast corner and the other on the southwest edge of the fire scar--are difficult to distinguish based on vegetation, topography, or climate from nearby burned areas but do correspond to areas that have burned since 2000.


## Live fuel moisture content

Live fuel moisture content (LFMC) is a measure of how dry/ignitable potential wildfire fuel is. The LFMC estimates plotted here were calculated for 1 Nov 2018 and were downloaded from a [Google Earth Engine app](https://kkraoj.users.earthengine.app/view/live-fuel-moisture) created using the neutral-network model described by Rao et al. (2020).

In [ ]:
# Plot LFMC data immediately preceding the Woolsey Fire
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
plot_bands(
    all_data["LFMC"],
    ax=ax,
    title=("Woolsey Fire - Live fuel moisture content - 30 m\n"
           "1 Nov 2018"),
    cmap="summer_r"
)
woolsey_fire.plot(ax=ax, edgecolor="black", facecolor="none", linewidth=2)

pass

**Description:** The LFMC plot shows generally dry conditions immediately preceding the Woolsey Fire. The resolution of the downloaded dat is coarse and the band of relatively healthy and moist vegetation observed in the FAL and spectral index plots is difficult to discern here.

## Histograms

I plotted these histograms to look for parameters where moderate- to high-severity burns did not occur that could be excluded from models. Each histogram plots pixel counts for each class on the MTBS classified dNBR dataset for a single parameter. Few pixels were classified as high-severity burns, so the plots use a log scale.

In [ ]:
plt.rc("figure.constrained_layout", use=True, h_pad=18/72)

# Plot histograms
keys = [
    "FAL",
    "LFMC",
    "NDMI",
    "NDVI",
    "NDWI",
    "SAVI",
    "Slope",
    "Folded Aspect",
    "Elevation",
]

# Set masks for different burn severities
any_sev = np.where(all_data["Classified dNBR"] > 1, True, False)
low_sev = np.where(all_data["Classified dNBR"].values == 2, True, False)
mod_sev = np.where(all_data["Classified dNBR"].values == 3, True, False)
high_sev = np.where(all_data["Classified dNBR"].values == 4, True, False)

# Set kwargs for the histogram on each axis
kwargs = {
    "log": True,
}

# Use the same colors for burn severity used by the MTBS plot
colors = ["#CCCCCC", "#7FFFD4", "#FFFF00", "#FF0000"]

# Plot histograms
fig, axes = plt.subplots(len(keys), 1, figsize=(8, 4 * len(keys)))
fig.suptitle("Histograms for moderate and high severity pixels")
for key, ax in zip(keys, axes):
    
    x = all_data[key]
    if len(x.shape) > 2:
        x = x[-1]
    
    for i, x in enumerate([
        np.sort(np.ravel(x.values[any_sev])),
        np.sort(np.ravel(x.values[low_sev])),
        np.sort(np.ravel(x.values[mod_sev])),
        np.sort(np.ravel(x.values[high_sev])),
    ]):
        if not i:
             _, bins, _ = ax.hist(x, bins=30, color=colors[i], **kwargs)
        else:
            ax.hist(x, bins=bins, color=colors[i], **kwargs)
        
        ax.set(xlabel=key, ylabel="Count")
        ax.legend(["all", "low", "mod", "high"])